googleドライブ

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


huggingface

In [2]:
!pip install huggingface_hub

from google.colab import userdata
from huggingface_hub import login

# Secretsからトークンを取得
HF_TOKEN = userdata.get('HF_TOKEN')
login(HF_TOKEN)

wandb

In [3]:
from google.colab import userdata
import wandb

# Colabシークレットから取得
wandb_api_key = userdata.get('WANDB_API_KEY')
wandb.login(key=wandb_api_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: puwaer (puwaer-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

ライブラリ

In [4]:
!pip uninstall unsloth -y
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --upgrade torch
!pip install --upgrade xformers

import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-9_d8r2de/unsloth_8ec450f465d8497bb97cc492b09b241a
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-9_d8r2de/unsloth_8ec450f465d8497bb97cc492b09b241a
  Resolved https://github.com/unslothai/unsloth.git to commit 2ff5dc1a8de1614994a275785b7b64fb4db8cb5d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 329.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 366.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 320.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 345.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 282.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.7/123.7 kB 307.5 MB/s eta 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 18.0 MB/s eta 0:00:00


パラメータ

In [5]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset


max_seq_length = 1024                                   # コンテキスト長
dtype = None                                            # Noneにしておけば自動で設定
load_in_4bit = True                                     # 量子化 True

model_id = "/content/drive/MyDrive/program/code_translation/base_model/llama-3.2-1B"
#model_id = "/content/drive/MyDrive/program/code_translation/base_model/llama-3.2-3B"
model_adapter = "llama-3.2-1B_lora_2"
#model_adapter = "llama_test_sample_data_3"

dataset = load_dataset("json", data_files="/content/drive/MyDrive/program/code_translation/data/multilingual_train_and_kyopro_sft.json")
#dataset = load_dataset("json", data_files="/content/drive/MyDrive/program/code_translation/data/multilingual_train_sft_0.json")


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_id,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    trust_remote_code=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Generating train split: 0 examples [00:00, ? examples/s]

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
/content/drive/MyDrive/program/code_translation/base_model/llama-3.2-1B does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,                                                         # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,                                                #デフォルト16
    lora_dropout = 0,                                               #デフォルト0
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,                                            #デフォルト3407
    use_rslora = False,
    loftq_config = None,
    max_seq_length = max_seq_length,
)


# 学習時のプロンプトフォーマットの定義
prompt = """### Instruction
{}
### Response
{}"""


EOS_TOKEN = tokenizer.eos_token # トークナイザーのEOSトークン（文末トークン）
def formatting_prompts_func(examples):
    input = examples["input"] # 入力データ
    output = examples["output"] # 出力データ
    text = prompt.format(input, output) + EOS_TOKEN # プロンプトの作成
    return { "formatted_text" : text, } # 新しいフィールド "formatted_text" を返す
pass

# # 各データにフォーマットを適用
dataset = dataset.map(
    formatting_prompts_func,
    num_proc= 4,                      # 並列処理数を指定
)

dataset

Unsloth 2025.3.19 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


Map (num_proc=4):   0%|          | 0/38526 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'input', 'output', 'input_language', 'output_language', 'formatted_text'],
        num_rows: 38526
    })
})

In [7]:
"""
training_arguments: 学習の設定

  - output_dir:
      -トレーニング後のモデルを保存するディレクトリ

  - per_device_train_batch_size:
      - デバイスごとのトレーニングバッチサイズ

  - per_device_eval_batch_size:
      - デバイスごとの評価バッチサイズ

  - gradient_accumulation_steps:
      - 勾配を更新する前にステップを積み重ねる回数

  - optim:
      - オプティマイザの設定

  - num_train_epochs:
      - エポック数

  - eval_strategy:
      - 評価の戦略 ("no"/"steps"/"epoch")

  - eval_steps:
      - eval_strategyが"steps"のとき、評価を行うstep間隔

  - logging_strategy:
      - ログ記録の戦略

  - logging_steps:
      - ログを出力するステップ間隔

  - warmup_steps:
      - 学習率のウォームアップステップ数

  - save_steps:
      - モデルを保存するステップ間隔

  - save_total_limit:
      - 保存しておくcheckpointの数

  - max_steps:
      - トレーニングの最大ステップ数

  - learning_rate:
      - 学習率

  - fp16:
      - 16bit浮動小数点の使用設定

  - bf16:
      - BFloat16の使用設定

  - group_by_length:
      -  入力シーケンスの長さによりバッチをグループ化 (トレーニングの効率化)

  - report_to:
      - ログの送信先 ("wandb"/"tensorboard"など)
"""
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset=dataset["train"],
    max_seq_length = max_seq_length,
    dataset_text_field="formatted_text",
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        num_train_epochs = 1,
        logging_steps = 10,
        warmup_steps = 10,
        save_steps=100,
        save_total_limit=2,
        max_steps=-1,
        learning_rate = 2e-4,                   #デフォルト2e-4
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        group_by_length=True,
        seed = 3407,                            #デフォルト3407
        output_dir = model_adapter,             #wandbのプロジェクトの名前
        report_to = "wandb",
    ),
)

Unsloth: Tokenizing ["formatted_text"] (num_proc=2):   0%|          | 0/38526 [00:00<?, ? examples/s]

学習

In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 38,526 | Num Epochs = 1 | Total steps = 2,408
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.038500
20,1.013400
30,0.985600
40,1.028700
50,1.062100
60,0.882100
70,0.945800
80,0.894100
90,0.909000
100,0.882700


保存

In [9]:
# huggingfaceにLoRAアダプタだけ保存
model.push_to_hub_merged(
    model_adapter,
    tokenizer=tokenizer,
    save_method="lora",
    token=HF_TOKEN,
    private=True
)

Unsloth: Saving LoRA adapters. Please wait...


adapter_model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Saved lora model to https://huggingface.co/llama-3.2-1B_lora_2


In [10]:
# google driveにLoRAアダプタだけ保存
model.save_pretrained(
    f"/content/drive/MyDrive/program/code_translation/model_adapter/{model_adapter}",
    save_method="lora"
)
tokenizer.save_pretrained(f"/content/drive/MyDrive/program/code_translation/model_adapter/{model_adapter}")


('/content/drive/MyDrive/program/code_translation/model_adapter/llama-3.2-1B_lora_2/tokenizer_config.json',
 '/content/drive/MyDrive/program/code_translation/model_adapter/llama-3.2-1B_lora_2/special_tokens_map.json',
 '/content/drive/MyDrive/program/code_translation/model_adapter/llama-3.2-1B_lora_2/tokenizer.json')

推論

In [11]:
import json

benchmark_dataset = "/content/drive/MyDrive/program/code_translation/data/multilingual_test_prompt_300.json"
#benchmark_dataset = "/content/drive/MyDrive/program/code_translation/data/multilingual_test_prompt_0.json"

# Read JSON file
with open(benchmark_dataset, "r", encoding='utf-8') as f:
    datasets = json.load(f)

# 学習したモデルを用いてタスクを実行
from tqdm import tqdm

# 推論するためにモデルのモードを変更
FastLanguageModel.for_inference(model)

results = []
for dt in tqdm(datasets):
    input = dt["input"]

    prompt = f"""### Instruction\n{input}\n### Response\n"""

    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    outputs = model.generate(**inputs, max_new_tokens=1024, use_cache=True, do_sample=False, repetition_penalty=1.2)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).split('\n### Response\n')[-1]
    #prediction = prediction.lstrip('\n')

    results.append({"id": dt["id"], "input": input, "output": prediction,"input_language": dt["input_language"],"output_language": dt["output_language"]})


100%|██████████| 300/300 [58:20<00:00, 11.67s/it]


In [12]:
# jsonlで保存
output_filename = f"llm_{model_adapter}_output.json"
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

drive_output_path = f"/content/drive/MyDrive/program/code_translation/output_data/llm_{model_adapter}_output.json"
with open(drive_output_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)